In [3]:
#imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.signal as signal #signal processing stuff (e.g. filters, hilbert transform, etc.)
from scipy.stats import linregress
import scipy
import struct
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import svgutils.transform as sg
import sys

#nelpy!
import nelpy as nel  #recommended import for nelpy
import nelpy.plotting as npl  #recommended import for the nelpy plotting library
import nelpy.io.trodes as neltro

#I saved my online ripple detection filter taps and stuff in a file....gonna import this for all analysis
import imp
ripple_filtering = imp.load_source('ripple_filtering', '/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/DataAnalysisScripts/ripple_filtering.py')

/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shayok/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#paper plot settings
sns.set(rc={'figure.figsize': (12, 4),'lines.linewidth': 1, 'font.size': 18, 'axes.labelsize': 16, 'axes.titlesize':18, 'legend.fontsize': 12, 'ytick.labelsize': 12, 'xtick.labelsize': 12 })
sns.set_style('white')
sns.set_color_codes(palette='colorblind')
#plots show up within jupyter for matplotlib
%matplotlib inline 

In [4]:
FS=3000
FS_system=30000

In [5]:
filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.timestamps.dat"
print("*****************Loading LFP Timestamps*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
timeStamps = np.fromfile(f, dtype=np.uint32)
timeStampsSeconds = (timeStamps-timeStamps[0])/FS_system
print("Done!")

filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.LFP_nt2ch1.dat" 
print("*****************Loading LFP Data*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
dataT2 = np.fromfile(f, dtype=np.int16)

filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.LFP_nt3ch1.dat"
print("*****************Loading LFP Data*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
dataT3 = np.fromfile(f, dtype=np.int16)

filePath = "/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/singleChan.LFP/singleChan.LFP_nt4ch1.dat"
print("*****************Loading LFP Data*****************")
f = open(filePath,'rb')
instr = f.readline()
while (instr != b'<End settings>\n') :
   print(instr)
   instr = f.readline()
print('Current file position', f.tell())
dataT4 = np.fromfile(f, dtype=np.int16)

#Decimate
start = 0
while(timeStamps[start]%10 != 0):
    start+=1
timeStamps = timeStamps[start::10]
dataT2 = dataT2[start::10]
dataT3 = dataT3[start::10]
dataT4 = dataT4[start::10]
timeStampsSeconds = timeStampsSeconds[start::10]
print(timeStamps[0])
print("Data decimated to match data sent to modules")

*****************Loading LFP Timestamps*****************
b'<Start settings>\n'
b'Description: LFP timestamps\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'Clock rate: 30000\n'
b'Decimation: 1\n'
b'Time_offset: 0\n'
b'Fields: <time uint32>\n'
Current file position 185
Done!
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'nTrode_ID: 2\n'
b'nTrode_channel: 1\n'
b'Clock rate: 30000\n'
b'Voltage_scaling: 0.195\n'
b'Decimation: 1\n'
b'First_timestamp: 19935\n'
b'Reference: on\n'
b'Low_pass_filter: 400\n'
b'Fields: <voltage int16>\n'
Current file position 294
*****************Loading LFP Data*****************
b'<Start settings>\n'
b'Description: LFP data for one channel\n'
b'Byte_order: little endian\n'
b'Original_file: singleChan.rec\n'
b'nTrode_ID: 3\n'
b'nTrode_channel: 1\n'
b'Clock rate: 30000\n'
b'Voltage_scaling: 0.195\n'
b'Deci

In [40]:
# Only run first time for data generation! This takes a while otherwise
smoothed_envelope2, rippleData2 = ripple_filtering.rippleBandFilterSimulated(dataT2, timeStampsSeconds, FS,ripple_filtering.bandpassFilterTaps,ripple_filtering.lowpassFilterTaps)
smoothed_envelope3, rippleData3 = ripple_filtering.rippleBandFilterSimulated(dataT3, timeStampsSeconds, FS,ripple_filtering.bandpassFilterTaps,ripple_filtering.lowpassFilterTaps)

# smoothed_envelope4, rippleData4 = ripple_filtering.rippleBandFilterSimulated(dataT4, timeStampsSeconds, FS,ripple_filtering.bandpassFilterTaps,ripple_filtering.lowpassFilterTaps)

np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelope_simulatedT3.out",smoothed_envelope3,fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelope_simulatedT2.out",smoothed_envelope2,fmt='%10.5f')

# np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelope_simulatedT4.out",smoothed_envelope4,fmt='%10.5f')

# smoothed_envelope2 = np.loadtxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelopeT2.out")
smoothed_envelope2, rippleData2 = ripple_filtering.rippleBandFilter(dataT2, timeStampsSeconds, FS=FS)
smoothed_envelope3, rippleData3 = ripple_filtering.rippleBandFilter(dataT3, timeStampsSeconds, FS=FS)

np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelopeT3.out",smoothed_envelope3,fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelopeT3.out",smoothed_envelope2,fmt='%10.5f')

# smoothed_envelope4, rippleData4 = ripple_filtering.rippleBandFilter(dataT4, timeStampsSeconds, FS=FS)
# np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelopeT4.out",smoothed_envelope4,fmt='%10.5f')

/home/shayok/anaconda3/lib/python3.5/site-packages/scipy/signal/signaltools.py:1336: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/shayok/anaconda3/lib/python3.5/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/shayok/anaconda3/lib/python3.5/site-packages/scipy/signal/signaltools.py:1333: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array

In [20]:
smoothed_envelope3, rippleData3 = ripple_filtering.rippleBandFilterSimulated(dataT3, timeStampsSeconds, FS,ripple_filtering.bandpassFilterTaps,ripple_filtering.lowpassFilterTaps)
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelope_simulatedT3.out",smoothed_envelope3,fmt='%10.5f')


In [19]:
smoothed_envelope2, rippleData2 = ripple_filtering.rippleBandFilterSimulated(dataT2, timeStampsSeconds, FS,ripple_filtering.bandpassFilterTaps,ripple_filtering.lowpassFilterTaps)
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/smoothed_envelope_simulatedT2.out",smoothed_envelope2,fmt='%10.5f')


In [41]:
np.savez_compressed("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT2.npz",smoothed_envelopeT2=smoothed_envelope2)
np.savez_compressed("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT3.npz",smoothed_envelopeT3=smoothed_envelope3)

In [6]:
smoothed_envelopeT2 = np.load("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT2.npz")['smoothed_envelopeT2']
smoothed_envelopeT3 = np.load("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT3.npz")['smoothed_envelopeT3']
# smoothed_envelopeT4 = np.load("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/figureData/Figure6/smoothed_envelopeT4.npz")['smoothed_envelopeT4']

In [76]:
#set threshold, find threshold crossings, and find ripple bounds
SecondaryThresholdSigma = 0
thresholdSigma = 5
lengthCriteria = 0.015 #15 ms

In [77]:
#Tetrode 2
thresholdT2 = np.mean(smoothed_envelopeT2) + thresholdSigma*np.std(smoothed_envelopeT2)

#rippleEvents contains indices of events in smoothed_envelope that meet threshold
rippleEventsT2, _ = ripple_filtering.thresholdCrossings(smoothed_envelopeT2, thresholdT2)
#remove events that don't meet length criteria
rippleEventsT2 = rippleEventsT2[rippleEventsT2[:,1] - rippleEventsT2[:,0] >= np.round(FS * lengthCriteria),:]

#find start and end of putative ripple aka when ripple band signal comes back down to mean
secondaryThresholdT2 = np.mean(smoothed_envelopeT2) + SecondaryThresholdSigma*np.std(smoothed_envelopeT2)
rippleBoundsT2, broaderMaxesT2 = ripple_filtering.thresholdCrossings(smoothed_envelopeT2, secondaryThresholdT2)

#find windows that match and where they match aka where in the larger bound window does the smaller bound fit
outerBoundaryIndicesT2 = np.searchsorted(rippleBoundsT2[:,0], rippleEventsT2[:,0])
outerBoundaryIndicesT2 = outerBoundaryIndicesT2 - 1 #subtract 1 since searchsorted returns index after where it belongs

#Find extended boundaries for ripple events by pairing to largere window
#   (Note that there may be repeats if the larger window contains multiple > 3SD sections)
rippleBoundsT2 = rippleBoundsT2[outerBoundaryIndicesT2,:]
rippleMaxesT2 = broaderMaxesT2[outerBoundaryIndicesT2]

# Now, since all that we care about are the larger windows, so we should get rid of repeats
_, unique_idxT2 = np.unique(rippleBoundsT2[:,0], return_index=True)
rippleBoundsT2 = rippleBoundsT2[unique_idxT2,:]
rippleMaxesT2 = rippleMaxesT2[unique_idxT2]
rippleEventsT2 = rippleEventsT2[unique_idxT2,:]

offlineRippleDetectionsT2 = np.zeros(smoothed_envelopeT2.size)
for i in range(0, rippleMaxesT2.size):
    offlineRippleDetectionsT2[(rippleBoundsT2[i][0]):(rippleBoundsT2[i][1])] = rippleMaxesT2[i]

In [78]:
#Tetrode 3
thresholdT3 = np.mean(smoothed_envelopeT3) + thresholdSigma*np.std(smoothed_envelopeT3)

#rippleEvents contains indices of events in smoothed_envelope that meet threshold
rippleEventsT3, _ = ripple_filtering.thresholdCrossings(smoothed_envelopeT3, thresholdT3)
#remove events that don't meet length criteria
rippleEventsT3 = rippleEventsT3[rippleEventsT3[:,1] - rippleEventsT3[:,0] >= np.round(FS * lengthCriteria),:]

#find start and end of putative ripple aka when ripple band signal comes back down to mean
secondaryThresholdT3 = np.mean(smoothed_envelopeT3) + SecondaryThresholdSigma*np.std(smoothed_envelopeT3)
rippleBoundsT3, broaderMaxesT3 = ripple_filtering.thresholdCrossings(smoothed_envelopeT3, secondaryThresholdT3)

#find windows that match and where they match aka where in the larger bound window does the smaller bound fit
outerBoundaryIndicesT3 = np.searchsorted(rippleBoundsT3[:,0], rippleEventsT3[:,0])
outerBoundaryIndicesT3 = outerBoundaryIndicesT3- 1 #subtract 1 since searchsorted returns index after where it belongs

#Find extended boundaries for ripple events by pairing to largere window
#   (Note that there may be repeats if the larger window contains multiple > 3SD sections)
rippleBoundsT3 = rippleBoundsT3[outerBoundaryIndicesT3,:]
rippleMaxesT3 = broaderMaxesT3[outerBoundaryIndicesT3]

# Now, since all that we care about are the larger windows, so we should get rid of repeats
_, unique_idxT3 = np.unique(rippleBoundsT3[:,0], return_index=True)
rippleBoundsT3 = rippleBoundsT3[unique_idxT3,:]
rippleMaxesT3 = rippleMaxesT3[unique_idxT3]
rippleEventsT3 = rippleEventsT3[unique_idxT3,:]

offlineRippleDetectionsT3 = np.zeros(smoothed_envelopeT3.size)
for i in range(0, rippleMaxesT3.size):
    offlineRippleDetectionsT3[(rippleBoundsT3[i][0]):(rippleBoundsT3[i][1])] = rippleMaxesT3[i]

In [79]:
def merge_ripple_epochs(epocha, epochb, ):
        """Finds intersection (overlap) between two sets of epoch arrays.
        Sampling rates can be different.

        TODO: verify if this requires a merged EpochArray to work properly?

        Parameters
        ----------
        epoch : nelpy.EpochArray
        boundaries : bool
            If True, limits start, stop to epoch start and stop.
        meta : dict, optional
            New dictionary of meta data for epoch ontersection.

        Returns
        -------
        intersect_epochs : nelpy.EpochArray
        """

        new_starts = []
        new_stops = []
        epoch_a = epocha.copy().merge()
        epoch_b = epochb.copy().merge()

        for aa in epoch_a.time:
            for bb in epoch_b.time:
                if (aa[0] <= bb[0] < aa[1]) and (aa[0] < bb[1] <= aa[1]): # b starts after a and is shorter
                    new_starts.append(aa[0])
                    new_stops.append(aa[1])
                elif (aa[0] < bb[0] < aa[1]) and (aa[0] < bb[1] > aa[1]): # b starts after a and is longer
                    new_starts.append(aa[0])
                    new_stops.append(bb[1])
                elif (aa[0] > bb[0] < aa[1]) and (aa[0] < bb[1] < aa[1]): # b starts before a and a is longer
                    new_starts.append(bb[0])
                    new_stops.append(aa[1])
                elif (aa[0] >= bb[0] < aa[1]) and (aa[0] < bb[1] >= aa[1]): # b starts same time or before a and b is longer or same time as a
                    new_starts.append(bb[0])
                    new_stops.append(bb[1])

        epoch_a._time = np.hstack(
            [np.array(new_starts)[..., np.newaxis],
                np.array(new_stops)[..., np.newaxis]])

        return epoch_a

In [80]:
rippleBounds = merge_ripple_epochs(nel.EpochArray(rippleBoundsT2),nel.EpochArray(rippleBoundsT3))
rippleBounds

<EpochArray at 0x7efc14f71e10: 492 epochs> of duration 3 days 11:21:53 hours

# save appropriately

In [81]:
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsStartTwoChan5SD.out",rippleBounds.starts,fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsEndTwoChan5SD.out",rippleBounds.stops,fmt='%10.5f')

In [10]:
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsStartT25SD.out",rippleBoundsT2[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsEndT25SD.out",rippleBoundsT2[:,1],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsStartT35SD.out",rippleBoundsT3[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsEndT35SD.out",rippleBoundsT3[:,1],fmt='%10.5f')

In [12]:
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsStartTwoChan3SD.out",rippleBounds[:,0],fmt='%10.5f')
np.savetxt("/home/shayok/Documents/Code/RippleDetectionAnalysis/Cavaradossi/paperData/twoChanAnalysis/rippleBoundsEndTwoChan3SD.out",rippleBounds[:,1],fmt='%10.5f')

<div class="alert alert-info">
The comment in cell 20 can easily be implemented in python but I just don't want to think and would rather use a nested loop...so it's gonna be done within the C++ functions voting two channel function! It just makes life easier. I think I have code that does the merging with `nelpy` but this is faster for me to prototype :)
</div>

# Call C++ analysis function
Compile with below! Note: I just like having all warnings shown really no reason to compile with those flags<br>
`g++ -o votingTwoOfThree -Wall -pedantic -std=c++11 -pthread voting2of3.cpp`<br> 
or <br>
`g++ -o votingTwoOfTwo -Wall -pedantic -std=c++11 -pthread voting2of2.cpp` <--- only one used in the paper<br>
*make sure single channel is compiled as well with below! Run it only after the two channel begins i*<br>
`g++ -o twoChanDefnSingleChanAnalysis -Wall -pedantic -std=c++11 -pthread singleChanAnalysis_twoChanDefn.cpp`

<div class="alert alert-warning">
The code here is changed to compile metrics for each one of these. File names have been changed for convenience...compile appropriate .cpp file. The file structure will need to be changed if anyone other than me is running this code
</div>